In [1]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd

from pymatgen.transformations.standard_transformations import SupercellTransformation

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

import os

In [2]:
fcc_filepath = "data/a2-bcc.cif"
fe_structure_fcc = from_file(structure_file=fcc_filepath)
fe_structure_fcc[:] ="Fe" 
print(fe_structure_fcc)

Full Formula (Fe2)
Reduced Formula: Fe
abc   :   3.155000   3.155000   3.155000
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Fe    0    0    0
  1  Fe    0.5  0.5  0.5


In [10]:
N = 8
r = 4
fe_structure_supercell = fe_structure_fcc.copy()
fe_structure_supercell.make_supercell([N,N,N])
fe_neighbor_data = count_neighbors(fe_structure_supercell, r)
a = fe_neighbor_data.data_frame \
    .merge(fe_neighbor_data.data_frame.rename(columns={"j": "i", "i": "j"}), how="outer") \
    .sort_values(["i","distance_bin", "j"]) \
    .loc[:, ["i", "j", "distance_bin"]] \
    .reset_index(drop=True)

neighbors_per_site = a.groupby(["i"]).count().reset_index().rename(columns={"j": "count"}).loc[:, "count"].values
sites_per_distance_group = a.groupby(["i", "distance_bin"]).count().reset_index() \
                            .rename(columns={"j": "count"}).loc[:, "count"].values
neighbor_indices = a["j"].values


neighbor_count = np.dstack ( np.unique(sites_per_distance_group, return_index=True) )    
neighbor_count.dtype = np.dtype([('v', neighbor_count.dtype), ('i', neighbor_count.dtype)])
neighbor_count.sort(order='i', axis=1)
neighbor_count = neighbor_count.flatten()['v'].tolist()

print("Number of sites:", 2*N*N*N)
print("Number of 1st and 2nd neighbors for each index i:", neighbor_count)
print("Neighbor indices sorted over index i and distances: ", neighbor_indices)

Number of sites: 1024
Number of 1st and 2nd neighbors for each index i: [8, 6]
Neighbor indices sorted over index i and distances:  [ 512  519  568 ... 1015 1016 1022]


In [9]:
filename = os.getcwd()+'/lists/bcc_neighbor_lists_'+str(2*N*N*N)+'.txt'

n_sites = np.array([2*N*N*N])
f=open(os.getcwd()+"/lists/bcc_header",'w')
f.write("\n".join(" ".join(map(str, x)) for x in (n_sites,neighbor_count)))
f.close()
np.savetxt(filename, neighbor_indices,fmt='%d')

In [17]:
filename = os.getcwd()+'/lists/fe_1024_grid.txt'
np.savetxt(filename, fe_structure_supercell.frac_coords)

In [19]:
x,y,z = fe_structure_supercell.frac_coords[:,0], fe_structure_supercell.frac_coords[:,1], fe_structure_supercell.frac_coords[:,2] 



array([0.    , 0.125 , 0.25  , ..., 0.6875, 0.8125, 0.9375])